In [1]:
using Arrow
using CSV
using DataFrames
using FreqTables
using Pipe
using Statistics

In [2]:
ENV["LINES"], ENV["COLUMNS"] = 5, 200

(5, 200)

In [3]:
DATA  = "../../data"
df = CSV.File("$DATA/auto2.csv") |> DataFrame

,mpg,cylinders,displacement,horsepower,weight,acceleration,year,origin,name,brand
,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,String,String
1,18.0,8.0,307.0,130.0,3504.0,12.0,70.0,1.0,chevrolet chevelle malibu,chevrolet
2,15.0,8.0,350.0,165.0,3693.0,11.5,70.0,1.0,buick skylark 320,buick
3,18.0,8.0,318.0,150.0,3436.0,11.0,70.0,1.0,plymouth satellite,plymouth
4,16.0,8.0,304.0,150.0,3433.0,12.0,70.0,1.0,amc rebel sst,amc
5,17.0,8.0,302.0,140.0,3449.0,10.5,70.0,1.0,ford torino,ford
⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮


In [4]:
df2 = Arrow.Table("$DATA/auto2.arrow") |> DataFrame

,mpg,cylinders,displacement,horsepower,weight,acceleration,year,origin,name,brand
,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,String,String
1,18.0,8.0,307.0,130.0,3504.0,12.0,70.0,1.0,chevrolet chevelle malibu,chevrolet
2,15.0,8.0,350.0,165.0,3693.0,11.5,70.0,1.0,buick skylark 320,buick
3,18.0,8.0,318.0,150.0,3436.0,11.0,70.0,1.0,plymouth satellite,plymouth
4,16.0,8.0,304.0,150.0,3433.0,12.0,70.0,1.0,amc rebel sst,amc
5,17.0,8.0,302.0,140.0,3449.0,10.5,70.0,1.0,ford torino,ford
⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮


In [5]:
df == df2

true

In [6]:
df2.mpg

392-element Array{Float64,1}:
 ⋮

In [7]:
df3 = copy(df2)

,mpg,cylinders,displacement,horsepower,weight,acceleration,year,origin,name,brand
,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,String,String
1,18.0,8.0,307.0,130.0,3504.0,12.0,70.0,1.0,chevrolet chevelle malibu,chevrolet
2,15.0,8.0,350.0,165.0,3693.0,11.5,70.0,1.0,buick skylark 320,buick
3,18.0,8.0,318.0,150.0,3436.0,11.0,70.0,1.0,plymouth satellite,plymouth
4,16.0,8.0,304.0,150.0,3433.0,12.0,70.0,1.0,amc rebel sst,amc
5,17.0,8.0,302.0,140.0,3449.0,10.5,70.0,1.0,ford torino,ford
⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮


In [8]:
gdf = groupby(df, :brand)

,mpg,cylinders,displacement,horsepower,weight,acceleration,year,origin,name,brand
,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,String,String
1,18.0,8.0,307.0,130.0,3504.0,12.0,70.0,1.0,chevrolet chevelle malibu,chevrolet
2,14.0,8.0,454.0,220.0,4354.0,9.0,70.0,1.0,chevrolet impala,chevrolet
3,15.0,8.0,400.0,150.0,3761.0,9.5,70.0,1.0,chevrolet monte carlo,chevrolet
4,28.0,4.0,140.0,90.0,2264.0,15.5,71.0,1.0,chevrolet vega 2300,chevrolet
5,17.0,6.0,250.0,100.0,3329.0,15.5,71.0,1.0,chevrolet chevelle malibu,chevrolet
⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮
,mpg,cylinders,displacement,horsepower,weight,acceleration,year,origin,name,brand
,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,String,String
1,36.0,4.0,120.0,88.0,2160.0,14.5,82.0,3.0,nissan stanza xe,nissan


In [9]:
gdf[("ford",)]

,mpg,cylinders,displacement,horsepower,weight,acceleration,year,origin,name,brand
,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,String,String
1,17.0,8.0,302.0,140.0,3449.0,10.5,70.0,1.0,ford torino,ford
2,15.0,8.0,429.0,198.0,4341.0,10.0,70.0,1.0,ford galaxie 500,ford
3,21.0,6.0,200.0,85.0,2587.0,16.0,70.0,1.0,ford maverick,ford
4,10.0,8.0,360.0,215.0,4615.0,14.0,70.0,1.0,ford f250,ford
5,19.0,6.0,250.0,88.0,3302.0,15.5,71.0,1.0,ford torino 500,ford
⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮


In [10]:
brand_mpg = combine(gdf, :mpg => mean)

,brand,mpg_mean
,String,Float64
1,chevrolet,20.4721
2,buick,19.1824
3,plymouth,21.7032
4,amc,18.0704
5,ford,19.475
⋮,⋮,⋮


In [11]:
combine(gdf, :mpg => mean => :mean_mpg)

,brand,mean_mpg
,String,Float64
1,chevrolet,20.4721
2,buick,19.1824
3,plymouth,21.7032
4,amc,18.0704
5,ford,19.475
⋮,⋮,⋮


In [12]:
sort!(brand_mpg, :mpg_mean, rev=true)

,brand,mpg_mean
,String,Float64
1,vw,39.0167
2,nissan,36.0
3,triumph,35.0
4,honda,33.7615
5,datsun,31.113
⋮,⋮,⋮


In [13]:
freqtable(df, :brand, :origin)

ArgumentError: [91mArgumentError: reducing over an empty collection is not allowed[39m

In [14]:
origbrand = @pipe df |>
    groupby(_, :brand) |>
    combine(_, :origin => x -> length(unique(x)))

,brand,origin_function
,String,Int64
1,chevrolet,1
2,buick,1
3,plymouth,1
4,amc,1
5,ford,1
⋮,⋮,⋮


In [15]:
extrema(origbrand.origin_function)

(1, 1)

In [16]:
origbrand2 = @pipe df |>
    groupby(_, [:origin, :brand]) |>
    combine(_, nrow)

,origin,brand,nrow
,Float64,String,Int64
1,1.0,chevrolet,43
2,1.0,buick,17
3,1.0,plymouth,31
4,1.0,amc,27
5,1.0,ford,48
⋮,⋮,⋮,⋮


In [17]:
originvbrand = unstack(origbrand2, :brand, :origin, :nrow)

,brand,1.0,3.0,2.0
,String,Int64?,Int64?,Int64?
1,chevrolet,43,missing,missing
2,buick,17,missing,missing
3,plymouth,31,missing,missing
4,amc,27,missing,missing
5,ford,48,missing,missing
⋮,⋮,⋮,⋮,⋮


In [18]:
coalesce.(originvbrand, 0)

,brand,1.0,3.0,2.0
,String,Int64,Int64,Int64
1,chevrolet,43,0,0
2,buick,17,0,0
3,plymouth,31,0,0
4,amc,27,0,0
5,ford,48,0,0
⋮,⋮,⋮,⋮,⋮


In [21]:
print(names(originvbrand))

["brand", "1.0", "3.0", "2.0"]

In [22]:
print(propertynames(originvbrand))

[:brand, Symbol("1.0"), Symbol("3.0"), Symbol("2.0")]

In [23]:
originvbrand."1.0"

37-element Array{Union{Missing, Int64},1}:
 ⋮

In [24]:
origbrand3 = @pipe df |>
    groupby(_, :origin) |>
    combine(_, :brand => x -> Ref(unique(x)))

,origin,brand_function
,Float64,Array…
1,1.0,"[""chevrolet"", ""buick"", ""plymouth"", ""amc"", ""ford"", ""pontiac"", ""dodge"", ""chevy"", ""hi"", ""mercury"", ""oldsmobile"", ""chrysler"", ""chevroelt"", ""capri"", ""cadillac""]"
2,3.0,"[""toyota"", ""datsun"", ""mazda"", ""toyouta"", ""maxda"", ""honda"", ""subaru"", ""nissan""]"
3,2.0,"[""volkswagen"", ""peugeot"", ""audi"", ""saab"", ""bmw"", ""opel"", ""fiat"", ""volvo"", ""renault"", ""vw"", ""mercedes-benz"", ""mercedes"", ""vokswagen"", ""triumph""]"


In [25]:
@pipe df |> groupby(_, :origin) |> combine(_, :brand => unique)

,origin,brand_unique
,Float64,String
1,1.0,chevrolet
2,1.0,buick
3,1.0,plymouth
4,1.0,amc
5,1.0,ford
⋮,⋮,⋮


In [27]:
for i in axes(origbrand3, 1)
    for j in i + 1:nrow(origbrand3)
        println("$i v $j => ", 
                intersect(origbrand3.brand_function[[i, j]]...))
    end
end

1 v 2 => String[]
1 v 3 => String[]
2 v 3 => String[]


In [28]:
flatten(origbrand3, :brand_function)

,origin,brand_function
,Float64,String
1,1.0,chevrolet
2,1.0,buick
3,1.0,plymouth
4,1.0,amc
5,1.0,ford
⋮,⋮,⋮
